In [7]:
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
from datetime import datetime
from lxml import html
import pandas as pd
import collections
import urllib.request
from tqdm import tqdm

In [98]:
#creating Houston zip code
houston_zipcode = [77002, 77003, 77004, 77005, 77006, 77007, 77008, 77009, 77010, 77011, 77012, 77013, 77014, 77016,
                   77017, 77018, 77019, 77020, 77021, 77022, 77023, 77025, 77026, 77027, 77028, 77030, 77031, 77033,
                   77034, 77035, 77036, 77038, 77040, 77041, 77042, 77043, 77045, 77046, 77047, 77048, 77050, 77051, 
                   77053, 77054, 77056, 77057, 77058, 77059, 77061, 77062, 77063, 77064, 77065, 77066, 77067, 77068, 
                   77069, 77070, 77071, 77072, 77073, 77074, 77075, 77076, 77077, 77078, 77079, 77080, 77081, 77082,
                   77084, 77085, 77086, 77087, 77088, 77090, 77091, 77092, 77093, 77094, 77095, 77096, 77098, 77099, 
                   77201, 77336, 77339, 77345, 77357, 77365, 77373, 77375, 77377, 77379, 77386, 77388, 77407, 77429, 
                   77433, 77447, 77449, 77478, 77484, 77489, 77493, 77504, 77506, 77587]

# san_antonio_zipcode = [78023, 78056, 78073, 78109, 78112, 78154, 78201, 78202, 78203, 78204, 78205, 78207, 78208, 
#                        78209, 78210, 78211, 78212, 78213, 78214, 78215, 78216, 78217, 78218, 78219, 78220, 78221, 
#                        78222, 78223, 78224, 78225, 78226, 78227, 78228, 78229, 78230, 78231, 78232, 78233, 78234, 
#                        78235, 78236, 78237, 78238, 78239, 78240, 78242, 78243, 78244, 78245, 78247, 78248, 78249, 
#                        78250, 78251, 78252, 78253, 78254, 78255, 78256, 78257, 78258, 78259, 78260, 78263, 78264, 
#                        78266]

# austin_zipcode = [78610, 78613, 78617, 78641, 78652, 78653, 78660, 78664, 78681, 78701, 78702, 78703, 78704, 78705, 
#                   78712, 78717, 78719, 78721, 78722, 78723, 78724, 78725, 78726, 78727, 78728, 78729, 78730, 78731, 
#                   78732, 78733, 78734, 78735, 78736, 78737, 78738, 78739, 78741, 78742, 78744, 78745, 78746, 78747, 
#                   78748, 78749, 78750, 78751, 78752, 78753, 78754, 78756, 78757, 78758, 78759]

# zipcode = houston_zipcode + san_antonio_zipcode + austin_zipcode

In [9]:
#getting home address for house listed in each zip code
home_address=[]
page=1
for code in tqdm(houston_zipcode):
    website = "https://www.har.com/zipcode_{0}/realestate/for_sale?page=1".format(code)
    main_req = requests.get(website)
    main_content = BeautifulSoup(main_req.content,'html.parser')
    page_list = main_content.find_all('a')
    page_link = [item.get('href') for item in page_list]
    pages=list(set([s for s in page_link if (s!=None and "/realestate/for_sale?for_sale=1&zip_code" in s)]))
    for page in pages:
        page_url= 'https://www.har.com/'+page
        page_req = requests.get(page_url)
        page_content = BeautifulSoup(page_req.content,'html.parser')
        content = page_content.find_all('a',class_="address")
        if len(content)>0:
            for item in content:
                address='https://www.har.com/'+item.get('href')
                home_address.append(address)

100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [20:14<00:00, 11.24s/it]


In [10]:
len(home_address)

15102

In [11]:
#list of all features for each house

column=['Listing Price:', 'Address:', 'City:', 'State:', 'Zip Code:', 'County:', 'Subdivision:', 'Legal Description:',
 'Property Type:', 'Bedrooms:', 'Baths:', 'Garage(s):', 'Stories:', 'Style:', 'Year Built:', 'Building Sqft.:',
 'Lot Size:', 'Maintenance Fee:', 'Living:', 'Family Room:', 'Dining:', 'Kitchen:', 'Breakfast:', 'Primary Bedroom:',
 'Bedroom:', 'Primary Bath:', 'Game Room:', 'Study/Library:', 'Media Room:', 'Room Description:', 'Fireplace:',
 'Countertop:', 'Floors:', 'Bedroom Desc:', 'Kitchen Desc:', 'Bathroom Description:', 'Heating:', 'Cooling:',
 'Connections:', 'Ice Maker:', 'Microwave:', 'Oven:', 'Compactor:', 'Dishwasher:', 'Range:', 'Disposal:', 'Energy Feature:',
 'Interior:', 'Roof:', 'Foundation:', 'Private Pool Desc:', 'Private Pool:', 'Exterior Type:', 'Lot Description:',
 'Garage Carport:', 'Controlled Access:', 'Water Sewer:', 'Unit Location:', 'Front Door:', 'Area Pool:',
 'Exterior:', 'Dwelling Type:', 'HOA Mandatory:', 'List Type:', 'Financing Considered:', 'Other Fees:',
 'Taxes w/o Exemp:', 'Tax Rate:', 'Subdivision Name:', 'County / Zip Code:', 'Single Family Properties:', 'Average Bedrooms:',
 'Average Baths:', 'Median Square Ft.:', 'Median Lot Square Ft.:', 'Median Year Built:', 'Median Appraised Value:',
 'Neighborhood Value Range:', 'Median Price / Square ft.:', 'Master Planned Community:',
 'Water Amenity:', 'Market Area Name:', 'Home For Sales:', 'Average List Price:', 'Average Square Ft.:',
 'Average Price/Square Ft.:', 'Home For Lease:', 'Average Lease:', 'Average Lease/Square Ft.:', 'Extra Room:',
 'Wine Room:', 'Carport Description:', 'Median Appraised Value / Square ft.:', 'Den:', 'Utility Room Desc:', 'Sunroom:',
    'Guest Suite:', 'Bath:', 'Garage Apartment:']

In [12]:
#getting all information for each house and save them in dictionary
home_dict = collections.defaultdict(list)
value_list=[]
label_list=[]
for link in tqdm(home_address):
    ind_home_dict = collections.defaultdict(list)
    r = requests.get(link)
    home_prop=BeautifulSoup(r.content,'html.parser')
    label= home_prop.find_all(class_='dc_label')
    value=home_prop.find_all(class_='dc_value')
    image_list = home_prop.find_all('a')
    image_link = [item.get('href') for item in image_list]
    image=[s for s in image_link if (s!=None and "https://photos.harstatic.com/" in s)]
    home_dict['image_link'].append(image)
    if len(label)==len(value):
        label_list = [label[i].text for i in range(len(label))]
        value_list = [value[i].text for i in range(len(value))]
    else:
        print(link)
    for i in range(len(label_list)):
        if label_list[i] in ind_home_dict:
            if not isinstance(ind_home_dict[label_list[i]], list):
                ind_home_dict[label_list[i]]=[ind_home_dict[label_list[i]]]
            ind_home_dict[label_list[i]].append(value_list[i])
        else:
            ind_home_dict[label_list[i]] = value_list[i]
    for item in column:
        if item in ind_home_dict:
            
            home_dict[item].append(ind_home_dict[item])
        else:
            home_dict[item].append(None)
        

100%|██████████████████████████████████████████████████████████████████████████| 15102/15102 [3:15:17<00:00,  1.29it/s]


In [13]:
#converting dictionary to dataframe
data=pd.DataFrame(home_dict)

In [14]:
data.tail()

,image_link,Listing Price:,Address:,City:,State:,Zip Code:,County:,Subdivision:,Legal Description:,Property Type:,...,Extra Room:,Wine Room:,Carport Description:,Median Appraised Value / Square ft.:,Den:,Utility Room Desc:,Sunroom:,Guest Suite:,Bath:,Garage Apartment:
15097,[https://photos.harstatic.com/190812338/hr/img...,"$ 189,000 ($111.77/sqft.) $Convert",1003 Longley Street,South Houston,TX,77587,Harris County,Merilyn Place Sec 03 (View subdivision price ...,LT 70 BLK 4 MERILYN PLACE SEC 3,Single-Family,...,None,None,Attached Carport,None,None,None,None,None,None,None
15098,[https://photos.harstatic.com/190340708/hr/img...,"$ 177,900 ($135.18/sqft.) $Convert",602 Missouri Street,South Houston,TX,77587,Harris County,South Houston (View subdivision price trend),LT 1 BLK 141 SOUTH HOUSTON,Single-Family,...,None,None,None,None,None,None,None,None,None,None
15099,[https://photos.harstatic.com/190401279/hr/img...,"$ 169,500 ($146.12/sqft.) $Convert",712 Minnesota Street,South Houston,TX,77587,Harris County,South Houston (View subdivision price trend),LT 6 BLK 128 SOUTH HOUSTON,Single-Family,...,None,None,None,None,None,None,None,None,None,None
15100,[https://photos.harstatic.com/189364347/hr/img...,"$ 165,000 ($145.37/sqft.) $Convert",612 Wisconsin Street,South Houston,TX,77587,Harris County,South Houston (View subdivision price trend),LT 6 BLK 137 SOUTH HOUSTON,Single-Family,...,None,None,None,None,None,None,None,None,"[1st, 1st]",None
15101,[https://photos.harstatic.com/190177640/hr/img...,"$ 155,000 ($184.74/sqft.) $Convert",705 Washington Street,South Houston,TX,77587,Harris County,South Houston (View subdivision price trend),LT 14 BLK 123 SOUTH HOUSTON,Single-Family,...,None,None,None,None,None,None,None,None,None,None


In [15]:
#saving to .CSV
data.to_csv('../House/Houston_Home_List.csv')

In [16]:
data.shape

(15102, 100)

In [104]:
tabel = []
header = ['Subdivision','Zip', 'Med.Appraisal','Avg.Sqft.', 'Avg.Yr.Built']
for code in tqdm(houston_zipcode):
    website = "https://www.har.com/zipcode_{0}/realestate".format(code)
    main_req = requests.get(website)
    main_content = BeautifulSoup(main_req.content,'html.parser')
    tab = main_content.find("div", {"class": "section_content hidden-phone"})
    if tab != None:
        for tr in tab.find_all('tr')[1:]:
            tds = tr.find_all('td')
            tabel.append({'Subdivision': tds[0].text,
                          'Zip': tds[1].string,
                          'Med.Appraisal': tds[2].text,
                          'Avg.Sqft.': tds[3].text,
                          'Avg.Yr.Built': tds[4].text
            }) 
              
    
#     page_link = [item.get('href') for item in page_list]
#     pages=list(set([s for s in page_link if (s!=None and "/realestate/for_sale?for_sale=1&zip_code" in s)]))
#     for page in pages:
#         page_url= 'https://www.har.com/'+page
#         page_req = requests.get(page_url)
#         page_content = BeautifulSoup(page_req.content,'html.parser')
#         content = page_content.find_all('a',class_="address")
#         if len(content)>0:
#             for item in content:
#                 address='https://www.har.com/'+item.get('href')
#                 home_address.append(address)

100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [06:57<00:00,  3.87s/it]


In [105]:
sub=pd.DataFrame(tabel)

In [106]:
sub.tail()

,Subdivision,Zip,Med.Appraisal,Avg.Sqft.,Avg.Yr.Built
3990,Christa Country,77587,"$87,739","1,020",1982
3991,Merilyn Place,77587,"$75,165","1,160",1950
3992,Pecan Terrace,77587,"$71,241","1,120",1949
3993,Merilyn Plaza,77587,"$66,733","1,159",1960
3994,Allen Genoa Condo,77587,"$30,851","1,116",1963


In [107]:
sub.to_csv('../House/Subdevision.csv')

In [ ]:
#downlaoding all images for each house
for idx in range(len(data.image_link)):
    for i in range(len(data.image_link[idx])):
        try:
            urllib.request.urlretrieve(data.image_link[idx][i][0], "../House/Har Image/{0}_image{1}.jpg".format(idx,i))
        except:
            None